Demo of land cover regridding module.

In [14]:
import xarray as xr
import numpy as np
import random
import pandas as pd
from src import land_cover

random.seed(10)
np.random.seed(10)

Create a dummy land cover dataset to be regridded.

In [2]:
# dummy land cover dataset
dummy_data = np.random.randint(3, size=(1, 4, 5))
lat = np.arange(1, 3, 0.5)
lon = np.arange(12, 14.5, 0.5)
time = pd.Timestamp("2020-09-01")

In [3]:
ds = xr.Dataset(
    data_vars=dict(
        lccs_class=(["time", "lat", "lon"], dummy_data),
    ),
    coords=dict(
        time=[time],
        lat=(["lat"], lat),
        lon=(["lon"], lon),
    ),
    attrs=dict(description="Dummy land cover data."),
)
ds

<xarray.Dataset>
Dimensions:     (time: 1, lat: 4, lon: 5)
Coordinates:
  * time        (time) datetime64[ns] 2020-09-01
  * lat         (lat) float64 1.0 1.5 2.0 2.5
  * lon         (lon) float64 12.0 12.5 13.0 13.5 14.0
Data variables:
    lccs_class  (time, lat, lon) int64 1 1 0 0 1 0 1 1 0 1 1 2 0 1 0 2 0 2 0 0
Attributes:
    description:  Dummy land cover data.

Create a dataset with target grid.

In [5]:
# dummy target grid
dummy_data = np.random.randint(12, size=(3, 2, 2))
lat = np.array([1.2, 2.1])
lon = np.array([12.2, 12.9])
time = pd.date_range("2020-09-06", periods=3)

In [6]:
ds_target = xr.Dataset(
    data_vars=dict(
        temperature=(["time", "lat", "lon"], dummy_data),
    ),
    coords=dict(
        time=time,
        lat=(["lat"], lat),
        lon=(["lon"], lon),
    ),
    attrs=dict(description="Dummy target dataset."),
)
ds_target

<xarray.Dataset>
Dimensions:      (time: 3, lat: 2, lon: 2)
Coordinates:
  * time         (time) datetime64[ns] 2020-09-06 2020-09-07 2020-09-08
  * lat          (lat) float64 1.2 2.1
  * lon          (lon) float64 12.2 12.9
Data variables:
    temperature  (time, lat, lon) int64 4 11 6 8 11 10 1 8 4 1 3 6
Attributes:
    description:  Dummy target dataset.

Now we can simply call the regrid function from `land_cover` to perform most common class selection.

In [15]:
regrid_land_cover = land_cover.regrid(ds, ds_target)
regrid_land_cover

<xarray.Dataset>
Dimensions:     (time: 1, lat: 2, lon: 2)
Coordinates:
  * time        (time) datetime64[ns] 2020-09-01
  * lat         (lat) float64 1.2 2.1
  * lon         (lon) float64 12.2 12.9
Data variables:
    lccs_class  (time, lat, lon) int64 1 0 2 0
Attributes:
    description:  Dummy land cover data.

Now let's verify the regridding of land cover classes. <br>
According to our implementation in `land_cover`, we can get the resolution of target grid and calculate target regridding intervals, which are used to `groupby` the data points from land cover dataset, based on both the resolution and the target grid.

In [16]:
(cell_lat_target, cell_lon_target) = land_cover.infer_resolution(ds_target)

lat_bounds = land_cover._construct_intervals(ds_target["lat"].values, cell_lat_target)
lon_bounds = land_cover._construct_intervals(ds_target["lon"].values, cell_lon_target)
print("interval by latitudes", lat_bounds)
print("interval by longitudes", lon_bounds)

interval by latitudes IntervalIndex([[0.7499999999999999, 1.65), [1.65, 2.55)], dtype='interval[float64, left]')
interval by longitudes IntervalIndex([[11.849999999999998, 12.55), [12.55, 13.25)], dtype='interval[float64, left]')


Next, we can group the data points on land cover grid by the intervals of regridding. Here we only show the grid points corresponding to lat[0] and lon[0] (by index) on new grid.

In [9]:
sel_ds_index_0_0 = ds["lccs_class"].sel(
    lat=slice(lat_bounds[0].left, lat_bounds[0].right),
    lon=slice(lon_bounds[0].left, lon_bounds[0].right),
)
sel_ds_index_0_0.values

array([[[1, 1],
        [0, 1]]])

Now we can pick up the most common label for this point!

In [11]:
expected_most_common_index_0_0 = land_cover._most_common_label(sel_ds_index_0_0.values)
expected_most_common_index_0_0

1

Finally we can verify our regridding.

In [17]:
assert regrid_land_cover["lccs_class"].values[0,0,0] == expected_most_common_index_0_0